In [6]:
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import scipy
from scipy import ndimage
import tensorflow as tf
from tensorflow.python.framework import ops
#from cnn_utils import *
import os
os.environ['CUDA_VISIBLE_DEVICES']='-1'
import deepdish.io as io
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, LeakyReLU
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score, cross_val_predict, train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.metrics import roc_curve, auc
from sklearn import metrics
from itertools import izip
import random

In [7]:
#Loading data
data_sig_A = io.load(os.path.join('MEM-A-dbsig.h5'))
data_sig_B = io.load(os.path.join('MEM-B-dbsig.h5'))
#data_sig_C = io.load(os.path.join('MEM-C-0500_unscaled_signal.h5'))
data_back_A = io.load(os.path.join('MEM-A-dbbkg.h5'))
data_back_B = io.load(os.path.join('MEM-B-dbbkg.h5'))
data_sig_B.keys()

['y', 'X', 'signal_lh']

In [8]:
#Assigning data
X_Big_train = data_sig_A['X']
target_BigSig_train = data_sig_A['signal_lh']
y_BigSig_train = data_sig_A['y']

X_Big_test = data_sig_B['X']
target_BigSig_test = data_sig_B['signal_lh']
y_BigSig_test = data_sig_B['y']

target_BigBack_train = data_back_A['bkg_lh']
y_BigBack_train = data_back_A['y']
target_BigBack_test = data_back_B['bkg_lh']
y_BigBack_test = data_back_B['y']

sig_to_backRatio_train = target_BigSig_train/target_BigBack_train
sig_to_backRatio_test = target_BigSig_test/target_BigBack_test

softTarget_train= target_BigSig_train/(target_BigSig_train+(0.1*target_BigBack_train))
softTarget_test = target_BigSig_test/(target_BigSig_test+(0.1*target_BigBack_test))

print np.shape(sig_to_backRatio_train)
print np.shape(sig_to_backRatio_test)
print target_BigBack_test.max()
print target_BigBack_test.min()

(520739,)
(173580,)
1.0765009091544848e-22
3.121791221788734e-48


In [12]:
def sequential_model():
    seqModel = Sequential()
    seqModel.add(Dense(32, input_dim=32, kernel_initializer='glorot_normal', activation='relu'))
    seqModel.add(Dense(12, kernel_initializer='glorot_normal', activation='relu'))
    seqModel.add(Dense(1, kernel_initializer='glorot_normal', activation='sigmoid'))
    seqModel.compile(loss='binary_crossentropy', optimizer='adam')
    return seqModel

In [13]:
sc_X = StandardScaler()
X_train1 = sc_X.fit_transform(X_Big_train)
X_test1 = sc_X.fit_transform(X_Big_test)
#X_train1 = tf.keras.utils.normalize(X_Big_train)
#X_test1 = tf.keras.utils.normalize(X_Big_test)
#np.shape(y_BigSig_train)
model= sequential_model()
#model.add(tf.keras.layers.Flatten())
model.fit(X_train1,y_BigSig_train, epochs=20, batch_size=256, verbose=2)

Epoch 1/20
 - 3s - loss: 0.1394
Epoch 2/20
 - 3s - loss: 0.0770
Epoch 3/20
 - 3s - loss: 0.0679
Epoch 4/20
 - 3s - loss: 0.0612
Epoch 5/20
 - 3s - loss: 0.0566
Epoch 6/20
 - 3s - loss: 0.0534
Epoch 7/20
 - 3s - loss: 0.0513
Epoch 8/20
 - 3s - loss: 0.0497
Epoch 9/20
 - 3s - loss: 0.0485
Epoch 10/20
 - 3s - loss: 0.0474
Epoch 11/20
 - 3s - loss: 0.0467
Epoch 12/20
 - 3s - loss: 0.0459
Epoch 13/20
 - 3s - loss: 0.0454
Epoch 14/20
 - 3s - loss: 0.0449
Epoch 15/20
 - 3s - loss: 0.0443
Epoch 16/20
 - 3s - loss: 0.0440
Epoch 17/20
 - 3s - loss: 0.0436
Epoch 18/20
 - 3s - loss: 0.0432
Epoch 19/20
 - 3s - loss: 0.0428
Epoch 20/20
 - 3s - loss: 0.0426
